In [57]:
import numpy as np 
import matplotlib.pyplot as plt
import cv2
import torchvision.models as models
import torch
from torchvision import datasets, transforms as T
import torch.nn as nn
from PIL import Image
from tqdm import tqdm
from matplotlib.patches import Circle, Rectangle
import matplotlib.ticker as plticker
import os



In [78]:
this_trial = 'trial_F'
crop_name = 'crop_45_55_20'

os.makedirs('../refinement_data/images_{}/{}'.format(crop_name, this_trial),exist_ok = True)
os.makedirs('../refinement_data/ratio_labels_{}/{}'.format(crop_name, this_trial),exist_ok = True)
os.makedirs('../refinement_data/pixel_labels_{}/{}'.format(crop_name, this_trial),exist_ok = True)

min_size = 45
max_size = 55
safe_margin = 20

random_generation = 10 ## number of random generation

In [79]:
image_path = os.path.join('../yolo_data/images',this_trial)

In [80]:
img_paths = []
label_paths = []
img_names = []
for this_image in os.listdir(image_path):
    img_paths.append(os.path.join(image_path,this_image))
    label_paths.append(os.path.join('../yolo_data/pixel_labels/',this_trial,'{}.npy'.format(this_image[:-5])))
    img_names.append(this_image[:-5])

In [81]:
for i, this_image in enumerate(tqdm(img_paths)):
    
    img = Image.open(this_image)
    label = np.load(label_paths[i])

    for j in range(label.shape[0]):
        
        gx, gy = label[j]

        for random_index in range(random_generation):

            rand_size_width = np.random.randint(min_size,max_size)
            ratio = 0.95+0.1*np.random.random()
            rand_size_height = int(rand_size_width*ratio)

            ax, ay = np.random.randint(safe_margin,rand_size_width-safe_margin), np.random.randint(safe_margin,rand_size_height-safe_margin)

            bx, by = rand_size_width -ax, rand_size_height -ay

            cropping_tuple = (gx-ax,gy-ay, gx+bx,gy+by)

            # print(cropped_tuple_left)

            cropped_img = img.crop(cropping_tuple)
            
            cropped_img.save(os.path.join('../refinement_data/images_{}/{}'.format(crop_name, this_trial),'{}_{}_{}.jpeg'.format(img_names[i],j,random_index)))

            ratio_labels = np.array([ax/rand_size_width, ay/rand_size_height])
            pixel_labels = np.array([ax,ay])
            np.save(os.path.join('../refinement_data/ratio_labels_{}/{}'.format(crop_name, this_trial),'{}_{}_{}.npy'.format(img_names[i],j,random_index)), ratio_labels)
            np.save(os.path.join('../refinement_data/pixel_labels_{}/{}'.format(crop_name, this_trial),'{}_{}_{}.npy'.format(img_names[i],j,random_index)), pixel_labels)

100%|██████████| 364/364 [00:03<00:00, 112.53it/s]


In [82]:
np.random.randint(24,60)

25